In [ ]:
# Set this equal to the directory containing only PTRF. 
ptrf_base_dir = '../filtered-Cav-PTRF_SRN/filtered-Cav-PTRF_568/'
cav_base_dir = '../filtered-Cav-PTRF_SRN/filtered-Cav-PTRF_647'

In [8]:
import os
import csv
from PIL import Image

def generate_combined_mask(ptrf_img_path, cav_img_path, combined_mask_path):
    # Load images
    ptrf_img = Image.open(ptrf_img_path).convert('1')  # Convert to binary
    cav_img = Image.open(cav_img_path).convert('1')  # Convert to binary

    assert ptrf_img.size == cav_img.size

    # Create a new image for the intersection mask
    combined_mask = Image.new('1', ptrf_img.size)

    # Access pixel data
    ptrf_pixels = ptrf_img.load()
    cav_pixels = cav_img.load()
    combined_pixels = combined_mask.load()

    area_combined = 0

    # Process pixels
    for x in range(combined_mask.width):
        for y in range(combined_mask.height):
            if ptrf_pixels[x, y] and cav_pixels[x, y]:
                combined_pixels[x, y] = 255
                area_combined += 1
            else:
                combined_pixels[x, y] = 0 
                
    area_cav = 0
    
    for x in range(combined_mask.width):
        for y in range(combined_mask.height):
            if cav_pixels[x, y]:
                area_cav += 1
    
    area_ptrf = 0
    
    for x in range(combined_mask.width):
        for y in range(combined_mask.height):
            if ptrf_pixels[x, y]:
                area_ptrf += 1

    # Save the combined mask
    combined_mask.save(combined_mask_path)

    return area_combined, area_cav, area_ptrf

def find_matching_files(ptrf_base_dir, cav_base_dir):
    ptrf_files = {}
    cav_files = {}

    # Walk through PTRF directory and store file paths
    for subdir, _, files in os.walk(ptrf_base_dir):
        for file in files:
            if file.endswith('bmask_class_0_Sep2022.bmp'):
                dir_num = os.path.basename(subdir)
                identifier = file.split('_PTRF_')[0]
                ptrf_files[(dir_num, identifier)] = os.path.join(subdir, file)
    
    # Walk through Cav directory and store file paths
    for subdir, _, files in os.walk(cav_base_dir):
        for file in files:
            if 'bmask_class_' in file and file.endswith('.bmp') and not "combined" in file:
                dir_num = os.path.basename(subdir)
                identifier = file.split('_Cav_')[0]
                class_num = file.split('bmask_class_')[-1].split('_')[0]
                cav_files[(dir_num, identifier, class_num)] = os.path.join(subdir, file)

    return ptrf_files, cav_files

def main():
    output_csv = 'mask_areas.csv'

    ptrf_files, cav_files = find_matching_files(ptrf_base_dir, cav_base_dir)

    # CSV header
    csv_header = ['Directory_Number', 'Identifier', 'Cav_Class', 'PTRF_File', 'Cav_File', 'Combined_Mask_File', 'Area_combined (pixels)', "Area_combined (um^2)", "Area_cav (pixels)", "Area_cav (um^2)", "Area_ptrf (pixels)", "Area_ptrf (um^2)"]

    # Open CSV file to write areas
    with open(output_csv, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(csv_header)

        # Iterate over each Cav class and match with PTRF files
        for (dir_num, identifier), ptrf_path in ptrf_files.items():
            for class_num in range(5):  # Cav classes 0-4
                if (dir_num, identifier, str(class_num)) in cav_files:
                    cav_path = cav_files[(dir_num, identifier, str(class_num))]
                    combined_mask_path = cav_path.replace('.bmp', '_combined.bmp')
                    print(cav_path, ptrf_path)
                    area, area_cav, area_ptrf = generate_combined_mask(ptrf_path, cav_path, combined_mask_path)
                    # (18/900)^2 = 0.0004
                    writer.writerow([dir_num, identifier, class_num, ptrf_path, cav_path, combined_mask_path, area, area*0.0004, area_cav, area_cav*0.0004, area_ptrf, area_ptrf*0.0004])

if __name__ == '__main__':
    main()


../filtered-Cav-PTRF_SRN/filtered-Cav-PTRF_647\1C8PTRF_Cav_PTRF_2\1\1C8PTRF_2_Cav_647_1_CC_filtered_final_aligned_c1_threshold_14_alpha_0.5_bmask_class_0_Sep2022.bmp ../filtered-Cav-PTRF_SRN/filtered-Cav-PTRF_568/1C8PTRF_Cav_PTRF_2\1\1C8PTRF_2_PTRF_568_1_CC_filtered_final_aligned_c2_threshold_14_alpha_1_bmask_class_0_Sep2022.bmp
../filtered-Cav-PTRF_SRN/filtered-Cav-PTRF_647\1C8PTRF_Cav_PTRF_2\1\1C8PTRF_2_Cav_647_1_CC_filtered_final_aligned_c1_threshold_14_alpha_0.5_bmask_class_1_Sep2022.bmp ../filtered-Cav-PTRF_SRN/filtered-Cav-PTRF_568/1C8PTRF_Cav_PTRF_2\1\1C8PTRF_2_PTRF_568_1_CC_filtered_final_aligned_c2_threshold_14_alpha_1_bmask_class_0_Sep2022.bmp
../filtered-Cav-PTRF_SRN/filtered-Cav-PTRF_647\1C8PTRF_Cav_PTRF_2\1\1C8PTRF_2_Cav_647_1_CC_filtered_final_aligned_c1_threshold_14_alpha_0.5_bmask_class_2_Sep2022.bmp ../filtered-Cav-PTRF_SRN/filtered-Cav-PTRF_568/1C8PTRF_Cav_PTRF_2\1\1C8PTRF_2_PTRF_568_1_CC_filtered_final_aligned_c2_threshold_14_alpha_1_bmask_class_0_Sep2022.bmp
../filt

In [12]:
generate_combined_mask(r"I:\filtered-Cav-PTRF_SRN\filtered-Cav-PTRF_568\1C8PTRF_Cav_PTRF_2\1\1C8PTRF_2_PTRF_568_1_CC_filtered_final_aligned_c2_threshold_14_alpha_1_bmask_class_0_Sep2022.bmp",
                       r"I:\filtered-Cav-PTRF_SRN\filtered-Cav-PTRF_647\1C8PTRF_Cav_PTRF_2\1\1C8PTRF_2_Cav_647_1_CC_filtered_final_aligned_c1_threshold_14_alpha_0.5_bmask_class_0_Sep2022.bmp",
                       r"..\filtered-Cav-PTRF_SRN\output.bmp",
                       r"..\filtered-Cav-PTRF_SRN\output.csv")

The area of the new mask is 12240 pixels.


12240